In [1]:
import matlab
import matlab.engine
import numpy as np
import tensorflow as tf
import keras
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Lambda

eng = matlab.engine.start_matlab()

Using TensorFlow backend.


In [45]:
np.random.seed(7)
X_train = np.loadtxt( "mu_train.txt", delimiter = "\n" )
Y_train = np.loadtxt( "exact_sol_train.txt", delimiter = "\t" )
#print(X_train)
#print(Y_train)
# print(len(Y_train))
# print(len(Y_train[0]))

model = Sequential()
model.add( Dense( 12, input_dim = 1, activation = "relu" ) )
model.add( Dense( 8, activation = "relu" ) )
model.add( Dense( 1, activation = "sigmoid" ) )
model.add( Lambda( lambda x: x*2 ) )

def my_lambda_layer(x):
    y = x
    print(x)
    for i in range(80):
        y = K.concatenate( [y, x], axis=1 )
        # print(y.shape)
    return y

model.summary()
    

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 12)                24        
_________________________________________________________________
dense_8 (Dense)              (None, 8)                 104       
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 9         
_________________________________________________________________
lambda_11 (Lambda)           (None, 1)                 0         
Total params: 137
Trainable params: 137
Non-trainable params: 0
_________________________________________________________________


In [50]:
def my_lambda_matlab_layer(x):
    
    init = tf.global_variables_initializer();
    sess = tf.Session();
    sess.run(init);
    x_ar=sess.run(x);
    x_mat=matlab.double(x_ar.tolist());
    y_mat=eng.lambda_fun(x_mat);
    y_ar=np.asarray(y_mat);
    y=tf.convert_to_tensor(y_ar, dtype=tf.float32);
    return y

ten=tf.constant([1]);
type(ten)
v=matlab_layer(ten);
type(v)



tensorflow.python.framework.ops.Tensor

In [51]:
def my_lambda_layer_output_shape(input_shape):
    return (input_shape[0],7)

model.add( Lambda( my_lambda_matlab_layer, output_shape = my_lambda_layer_output_shape ) )



model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'] )

model.fit( X_train, Y_train, epochs = 100, batch_size = 10 )

InvalidArgumentError: You must feed a value for placeholder tensor 'dense_7_input' with dtype float and shape [?,1]
	 [[node dense_7_input (defined at /Users/utente/anaconda3/envs/myenv/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517) ]]

Caused by op 'dense_7_input', defined at:
  File "/Users/utente/anaconda3/envs/myenv/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/utente/anaconda3/envs/myenv/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/utente/anaconda3/envs/myenv/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/utente/anaconda3/envs/myenv/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/utente/anaconda3/envs/myenv/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/Users/ute